In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Creazione del Client

In questo esempio, utilizzeremo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) per accedere al LLM.

Il `model` è definito come `gpt-4o-mini`. Prova a cambiare il modello con un altro disponibile nel marketplace di GitHub Models per vedere risultati diversi.

Come test rapido, eseguiremo semplicemente un prompt semplice - `Qual è la capitale della Francia`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definire le Funzioni

In questo esempio, daremo all'agente accesso a uno strumento che è una funzione con un elenco di destinazioni di vacanza disponibili e la loro disponibilità.

Puoi immaginare che questo sia uno scenario in cui un agente di viaggio potrebbe avere accesso, ad esempio, a un database di viaggi.

Mentre segui questo esempio, sentiti libero di provare a definire nuove funzioni e strumenti che l'agente può utilizzare.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definizione dello strumento funzione
Per far sì che l'agente utilizzi `vacation_destinations` come `FunctionTool`, dobbiamo definirlo come tale.

Forniremo anche una descrizione dello strumento, utile per l'agente per identificare a cosa serve quello strumento in relazione al compito richiesto dall'utente.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definire l'agente

Ora possiamo creare l'agente nel codice qui sotto. Definiamo il `system_message` per fornire istruzioni all'agente su come aiutare gli utenti a trovare destinazioni per le vacanze.

Impostiamo anche il parametro `reflect_on_tool_use` su true. Questo permette di utilizzare l'LLM per prendere la risposta della chiamata allo strumento e inviarla utilizzando un linguaggio naturale.

Puoi impostare il parametro su false per vedere la differenza.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Esecuzione dell'Agente

Ora possiamo eseguire l'agente con il messaggio iniziale dell'utente che chiede di organizzare un viaggio a Tokyo.

Puoi cambiare questa destinazione per vedere come l'agente risponde alla disponibilità della città.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
